In [2]:
import pandas as pd
import ast

In [5]:
# Training queries
MedQA_queries = pd.read_csv("/scratch/s190619/Data_etc/MedQA/disorders_table_train.csv")
MedQA_queries.head()

,qid,query,answer,options,meta_info,Disorder,CUI,TUI,short_category,long_category,description,cui_METAMAP
0,3,A pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,"{'A': 'Thromboembolism', 'B': 'Pulmonary ische...",step1,True,C0040038,T046,DISO,Disorders,Pathologic Function,['C0040038']
1,4,A 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,"{'A': 'Factor V Leiden', 'B': 'Hemophilia A', ...",step1,True,C0042974,T047,DISO,Disorders,Disease or Syndrome,['C0042974']
2,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0238417,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
3,5,A 40-year-old zookeeper presents to the emerge...,Scorpion sting,"{'A': 'Aspirin', 'B': 'Oral contraceptive pill...",step1,True,C0261663,T037,DISO,Disorders,Injury or Poisoning,['C0238417']
4,21,An investigator is studying the function of th...,Anorexia,"{'A': 'Hypothermia', 'B': 'Hyperthermia', 'C':...",step1,True,C0003123,T047,DISO,Disorders,Disease or Syndrome,"['C0003123', 'C1971624']"


In [6]:
def clean_string(x):
    return x.replace("\t"," ").replace("\n"," ")

q = MedQA_queries.copy()
q["options"] = q["options"].apply(lambda x: ast.literal_eval(x))
opts = []
for i in range(q["options"].shape[0]):
    opt_tmp = [x for x in q["options"].iloc[i].values() if x != q["answer"].iloc[i]]
    opt_tmp.insert(0, q["answer"].iloc[i])
    opts.append(opt_tmp)
q["options"] = opts 
q = q[["query","options"]]
for i in range(5):
    q[f"options{i}"] = [clean_string(q.iloc[j]["options"][i]) for j in range(q.shape[0])]
q = q.drop("options",axis=1)
q["query"] = [x.lower() for x in q["query"]]
q = q.drop_duplicates("query")

In [7]:
q.head()

,query,options0,options1,options2,options3,options4
0,a pulmonary autopsy specimen from a 58-year-ol...,Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage
1,a 20-year-old woman presents with menorrhagia ...,Von Willebrand disease,Factor V Leiden,Hemophilia A,Lupus anticoagulant,Protein C deficiency
2,a 40-year-old zookeeper presents to the emerge...,Scorpion sting,Aspirin,Oral contraceptive pills,Hypothyroidism,Obesity
4,an investigator is studying the function of th...,Anorexia,Hypothermia,Hyperthermia,Polydipsia,Nocturnal hyperactivity
7,a 42-year-old woman is in the hospital recover...,Delirium tremens,Acute cholangitis,Alcoholic hallucinosis,Hepatic encephalopathy,Thyroid storm


# Create MedQA triples with options

In [8]:
triples_MedQA = pd.read_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/MedQA_textbooks_split_triples.tsv", sep="\t", header=None)
triples_MedQA.columns = ["query","positive","negative"]

In [9]:
triples_with_options_MedQA = triples_MedQA.join(q.set_index("query"), on="query")

In [10]:
triples_with_options_MedQA.isna().sum()

query       0
positive    0
negative    0
options0    0
options1    0
options2    0
options3    0
options4    0
dtype: int64

In [11]:
triples_with_options_MedQA.to_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/MedQA_textbooks_split_triples_w_options.tsv", sep="\t", header = False, index = False)

# Create FZ triples with options

In [12]:
triples_FZ = pd.read_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/FZ_corpus_split_triples.tsv", sep="\t", header=None)
triples_FZ.columns = ["query","positive","negative"]

In [13]:
triples_with_options_FZ = triples_FZ.join(q.set_index("query"), on="query")

In [14]:
triples_with_options_FZ.isna().sum()

query       0
positive    0
negative    0
options0    0
options1    0
options2    0
options3    0
options4    0
dtype: int64

In [43]:
triples_with_options_FZ.to_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/FZ_corpus_split_triples_w_options.tsv", sep="\t", header = False, index = False)

In [41]:
triples_with_options_MedQA.head()

,query,positive,negative,options0,options1,options2,options3,options4
0,a pulmonary autopsy specimen from a 58-year-ol...,the anticoagulant effect of lmwhs may be parti...,essential tremor may cause difficulty in diagn...,Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage
1,a pulmonary autopsy specimen from a 58-year-ol...,the longitudinal assessment of bariatric surge...,gut-associated lymphoid tissues (galt) lymph...,Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage
2,a pulmonary autopsy specimen from a 58-year-ol...,a randomized controlled trial involving 246 lo...,an autoimmune disorder caused by antibodies di...,Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage
3,a pulmonary autopsy specimen from a 58-year-ol...,makary et al studied 594 older patients presen...,"baerwald ar, walker ra, pierson ra. growth rat...",Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage
4,a pulmonary autopsy specimen from a 58-year-ol...,with the clini-cal history and physical examin...,few trials have evaluated exercise speciically...,Thromboembolism,Pulmonary ischemia,Pulmonary hypertension,Pulmonary passive congestion,Pulmonary hemorrhage


In [19]:
triples_with_options_MedQA.sample(frac=1).reset_index().drop("index",axis=1).head()

,query,positive,negative,options0,options1,options2,options3,options4
0,a 55-year-old man presents to the emergency de...,deeply infiltrative lesions are consistently a...,"seidman, j.g., and leder, p.: the arrangement ...",Adhesions,Enteric nervous system damage,Impacted stool,Norovirus,Twisting of the bowel
1,a 38-year-old woman with a history of crohn’s ...,the vasoac-tive intestinal peptide-secreting t...,"feldmann, m., and maini, r.n.: lasker clinical...",Hypokalemia,Hyperkalemia,Normal random blood glucose levels,Metabolic acidosis,Hypoglycemia
2,a 66-year-old man is brought to the emergency ...,"occasionally, atrophy of bone and pain in the ...",infants with tricuspid atresia are usually sev...,Shoulder pain,Pain radiating to the back,Gross hematuria,Muffled heart sounds,Free air on chest radiograph
3,"a 30-year-old woman, gravida 2, para 1, at 38 ...","oral, direct inhibitors of fxa have been appro...","wang, j., and eck, m.j.: assembling atomic res...",Routine monitoring,Vibroacoustic stimulation,Placement of fetal scalp electrode,Emergent cesarean delivery,Administer tocolytics
4,a 15-year-old boy is brought to the physician ...,"the disruptive, impulse—control, and conduct d...",figure 24-24 a. st segment changes in normal a...,Conduct disorder,Autism spectrum disorder,Oppositional defiant disorder,Antisocial personality disorder,"Disruptive mood dysregulation disorder """


In [20]:
triples_with_options_MedQA.shape

(40069, 8)

In [21]:
triples_with_options_MedQA.sample(frac=1).reset_index().drop("index",axis=1).shape

(40069, 8)

In [25]:
triples_MedQA = pd.read_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/FZ_corpus_split_triples.tsv", sep="\t", header=None)
#triples_MedQA.columns = ["query","positive","negative"]
triples_MedQA.sample(frac=1).reset_index().drop("index",axis=1).to_csv("/scratch/s190619/Data_etc/ColBERT/training_sets/FZ_corpus_split_triples_scrambled.tsv", sep="\t",header=False,index=False)